### Since file is too large, I am doing eda for one sample company

In [2]:
import re

import pandas as pd

In [3]:
target_file = '/home/sjb/Projects/Research/LinkedIn_OB/data/company_level_individual_stay_term/ibm_person_stay_term.csv'

In [4]:
liwc_df = pd.read_pickle('./ibm_liwc_sample.pkl')
target_df = pd.read_csv(target_file)

In [5]:
liwc_df.shape, target_df.shape

((54169, 193), (54169, 3))

## 1. Sanity Check - person ids in liwc file and target file match

In [6]:
liwc_df.head()

,Unnamed: 0,company,Filename.x,Segment.x,WC.x,Analytic.x,Clout.x,Authentic.x,Tone.x,WPS.x,...,Comma.y,Colon.y,SemiC.y,QMark.y,Exclam.y,Dash.y,Quote.y,Apostro.y,Parenth.y,OtherP.y
1759259,1759260,ibm,ibm__26505646_531_First_1000_1.txt,1,69,97.77,55.76,80.16,78.80,23.00,...,9.07,0.62,0.27,0.01,0.03,2.16,1.96,0.41,1.43,3.36
1759260,1759261,ibm,ibm__26492667_530_First_1000_1.txt,1,112,95.40,63.97,10.26,87.20,22.40,...,9.07,0.62,0.27,0.01,0.03,2.16,1.96,0.41,1.43,3.36
1759261,1759262,ibm,ibm__26709630_535_First_1000_1.txt,1,49,85.53,72.97,1.69,91.78,12.25,...,9.07,0.62,0.27,0.01,0.03,2.16,1.96,0.41,1.43,3.36
1759262,1759263,ibm,ibm__26501938_531_First_1000_1.txt,1,89,97.56,63.20,1.50,94.47,44.50,...,9.07,0.62,0.27,0.01,0.03,2.16,1.96,0.41,1.43,3.36
1759263,1759264,ibm,ibm__26503564_531_First_1000_1.txt,1,55,91.40,35.79,77.10,25.77,27.50,...,9.07,0.62,0.27,0.01,0.03,2.16,1.96,0.41,1.43,3.36


In [8]:
REG_BEGIN_MATCH = re.compile('^.*__')
REG_END_MATHCH = re.compile('\.txt$')

In [9]:
def _transform_text(text):
    sub = re.sub(REG_BEGIN_MATCH, '', text)
    sub = re.sub(REG_END_MATHCH, '', sub)
    
    return sub

In [10]:
liwc_df_person_ids = set(liwc_df['Filename.x'].apply(_transform_text))
target_df_person_ids = set(target_df['person_id'])

check_diff = liwc_df_person_ids.symmetric_difference(target_df_person_ids)
check_diff

set()

In [14]:
assert not len(check_diff)

## 2. Clean LIWC DF

In [16]:
liwc_df_cols = liwc_df.columns

x_columns = liwc_df_cols[liwc_df_cols.str.endswith('.x')]
y_columns = liwc_df_cols[liwc_df_cols.str.endswith('.y')]

In [21]:
x_columns_truncated = x_columns.str.replace('\.x', '')
y_columns_truncated = y_columns.str.replace('\.y', '')

set(x_columns_truncated).symmetric_difference(y_columns_truncated)

set()

In [24]:
TO_EXCLUDE = [
    'Filename', 
    'Segment', 
    'WC'
]

In [25]:
clean_column_names = x_columns_truncated.difference(TO_EXCLUDE)

In [27]:
clean_column_names

Index(['AllPunc', 'Analytic', 'Apostro', 'Authentic', 'Clout', 'Colon',
       'Comma', 'Dash', 'Dic', 'Exclam', 'OtherP', 'Parenth', 'Period',
       'QMark', 'Quote', 'SemiC', 'Sixltr', 'Tone', 'WPS', 'achieve', 'adj',
       'adverb', 'affect', 'affiliation', 'anger', 'anx', 'article', 'assent',
       'auxverb', 'bio', 'body', 'cause', 'certain', 'cogproc', 'compare',
       'conj', 'death', 'differ', 'discrep', 'drives', 'family', 'feel',
       'female', 'filler', 'focusfuture', 'focuspast', 'focuspresent',
       'friend', 'function.', 'health', 'hear', 'home', 'i', 'informal',
       'ingest', 'insight', 'interrog', 'ipron', 'leisure', 'male', 'money',
       'motion', 'negate', 'negemo', 'netspeak', 'nonflu', 'number', 'percept',
       'posemo', 'power', 'ppron', 'prep', 'pronoun', 'quant', 'relativ',
       'relig', 'reward', 'risk', 'sad', 'see', 'sexual', 'shehe', 'social',
       'space', 'swear', 'tentat', 'they', 'time', 'verb', 'we', 'work',
       'you'],
      dtype=

In [29]:
clean_x_columns = clean_column_names + '.x'
clean_y_columns = clean_column_names + '.y'

In [32]:
liwc_df[clean_x_columns]

,AllPunc.x,Analytic.x,Apostro.x,Authentic.x,Clout.x,Colon.x,Comma.x,Dash.x,Dic.x,Exclam.x,...,social.x,space.x,swear.x,tentat.x,they.x,time.x,verb.x,we.x,work.x,you.x
1759259,36.23,97.77,0.00,80.16,55.76,0.00,8.70,11.59,71.01,0.00,...,4.35,15.94,0.0,1.45,0.00,4.35,7.25,0.00,13.04,0.00
1759260,33.04,95.40,0.00,10.26,63.97,1.79,3.57,1.79,75.89,0.00,...,3.57,8.04,0.0,0.00,0.00,0.89,2.68,0.00,32.14,0.00
1759261,30.61,85.53,0.00,1.69,72.97,2.04,6.12,0.00,79.59,0.00,...,6.12,2.04,0.0,0.00,0.00,0.00,2.04,0.00,34.69,0.00
1759262,25.84,97.56,0.00,1.50,63.20,0.00,12.36,5.62,56.18,0.00,...,3.37,6.74,0.0,0.00,0.00,1.12,0.00,0.00,10.11,0.00
1759263,14.55,91.40,0.00,77.10,35.79,0.00,5.45,0.00,80.00,1.82,...,3.64,5.45,0.0,0.00,0.00,9.09,7.27,0.00,10.91,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813423,18.18,92.14,0.00,46.36,41.03,0.00,9.09,0.00,77.27,0.00,...,3.41,6.82,0.0,0.00,0.00,3.41,4.55,0.00,10.23,0.00
1813424,14.56,97.98,0.00,93.51,24.82,0.00,6.80,0.97,77.67,0.00,...,0.97,11.65,0.0,0.00,0.00,6.80,6.80,0.00,11.65,0.00
1813425,11.65,67.77,0.57,17.28,95.83,0.85,1.70,0.28,94.89,0.57,...,18.18,8.81,0.0,2.56,0.28,5.11,15.62,1.14,9.66,4.26
1813426,12.70,97.40,0.00,68.01,62.46,0.00,3.17,0.00,82.54,0.00,...,7.94,12.70,0.0,0.00,0.00,3.17,7.94,1.59,9.52,0.00


## 3. Check y columns - mean?

In [40]:
clean_x_col_df = liwc_df[clean_x_columns]
clean_y_col_df = liwc_df[clean_y_columns]

In [39]:
(clean_y_col_df - clean_y_col_df.iloc[0]).abs().max().max()

0.0

In [42]:
clean_x_col_df.mean(axis=0)

AllPunc.x      24.833839
Analytic.x     93.069146
Apostro.x       0.351219
Authentic.x    28.032447
Clout.x        62.266307
                 ...    
time.x          2.861314
verb.x          4.693491
we.x            0.150853
work.x         19.382973
you.x           0.094688
Length: 92, dtype: float64

In [44]:
clean_y_col_df.iloc[0]

AllPunc.y      24.33
Analytic.y     95.15
Apostro.y       0.41
Authentic.y    20.88
Clout.y        63.75
               ...  
time.y          2.88
verb.y          4.99
we.y            0.18
work.y         18.57
you.y           0.12
Name: 1759259, Length: 92, dtype: float64

In [48]:
temp = clean_x_col_df / clean_x_col_df.std(axis=0)